In [2]:
import json
import boto3
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

EXTRACTING DATA FROM S3 BUCKET

In [5]:
s3 = boto3.client('s3')
print("Getting S3 object data...")
response = s3.get_object(Bucket='jsonbucket-usdata', Key='StoreSales.json')
data = response['Body'].read()
print("Done, data loaded into memory")
json_data = json.loads(data)

Getting S3 object data...
Done, data loaded into memory


TRANSFORMING THE DATA

In [6]:
# Converting json data into DataFrame as table format
df = pd.DataFrame(json_data)
# Removing null values
df.dropna(inplace=True)
#changing the column names
df.columns = df.columns.str.lower().str.replace(' ', '_')
# setting datatypes to the cloumns
df = df.astype({
    'row_id': 'int64',
    'order_id': 'string',
    'order_date': 'datetime64[ns]',
    'ship_date': 'datetime64[ns]',
    'ship_mode': 'string',
    'customer_id': 'string',
    'customer_name': 'string',
    'segment': 'category',
    'city': 'string',
    'state': 'string',
    'country': 'string',
    'postal_code': 'string',
    'market': 'string',
    'region': 'string',
    'product_id': 'string',
    'category': 'category',
    'sub-category': 'category',
    'product_name': 'string',
    'sales': 'float64',
    'quantity': 'int64',
    'discount': 'float64',
    'profit': 'float64',
    'shipping_cost': 'float64',
    'order_priority': 'category'
})

df['postal_code'] = df['postal_code'].replace('', np.nan)


In [8]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          51290 non-null  int64         
 1   order_id        51290 non-null  string        
 2   order_date      51290 non-null  datetime64[ns]
 3   ship_date       51290 non-null  datetime64[ns]
 4   ship_mode       51290 non-null  string        
 5   customer_id     51290 non-null  string        
 6   customer_name   51290 non-null  string        
 7   segment         51290 non-null  category      
 8   city            51290 non-null  string        
 9   state           51290 non-null  string        
 10  country         51290 non-null  string        
 11  postal_code     9994 non-null   string        
 12  market          51290 non-null  string        
 13  region          51290 non-null  string        
 14  product_id      51290 non-null  string        
 15  categor

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-05-02,2013-07-02,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-05-11,2013-06-11,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


LOAD INTO POSTGRESQL

In [57]:
# Define PostgreSQL connection parameters
database_username = 'postgres'
database_password = 'rootuser'
database_ip = '127.0.0.1'  # or your DB server IP
database_name = 'postgres'
database_port = '5432'  # Default PostgreSQL port

# Create the connection string
connection_string = f'postgresql+psycopg2://{database_username}:{database_password}@{database_ip}:{database_port}/{database_name}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Push the DataFrame to PostgreSQL (replace 'your_table_name' with the table name you want)
df.to_sql('saledata', engine, if_exists='replace', index=False)

print("Data pushed successfully to PostgreSQL!")

Data pushed successfully to PostgreSQL!
